In [ ]:
!pip install dall_e
!pip install datasets
!pip install transformers
!pip install tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires 

In [ ]:
import os
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
from torch.utils.data import Dataset
import random
from torchvision.datasets import CIFAR10
from torchvision import transforms as T
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
from torch.optim import Adam

In [ ]:
# '''class BEITDataset(Dataset):
#     def __init__(self, tokenizer, paths, mask_ratio=0.15, patches=196, image_size=224):
#         self.paths = paths
#         self.tokenizer = tokenizer
#         self.mask_ratio = mask_ratio
#         self.patches = patches
#         self.image_size = image_size
#         self.mask_token = nn.Parameter(torch.zeros(1, 1, embed_size))

#     def __len__(self):
#         return len(self.paths)

#     def getMasked(self, image, masking_ratio=0.05):
#         h, w = image.shape[2], image.shape[3]
#         patch_size = 16
#         num_patches_h = h // patch_size
#         num_patches_w = w // patch_size
#         num_masks = int(masking_ratio*num_patches_h*num_patches_w)
#         mask_positions = set()
#         while len(mask_positions) < num_masks:
#             x = random.randint(0, num_patches_h - 1)
#             y = random.randint(0, num_patches_w - 1)
#             mask_positions.add((x, y))
#         bool_masked_pos = torch.zeros(num_patches_h * num_patches_w, dtype=torch.bool)
#         for (x, y) in mask_positions:
#             idx = x * num_patches_w + y
#             bool_masked_pos[idx] = True
#         tokens = tokenize_image(image)
#         masked_image = image.clone()
#         masked_tokens = tokens.clone()
#         for (x, y) in mask_positions:
#             masked_image[:,:, x * patch_size:(x + 1) * patch_size, y * patch_size:(y + 1) * patch_size] = 0
#             masked_tokens[:,:, x * patch_size:(x + 1) * patch_size, y * patch_size:(y + 1) * patch_size] = self.mask_token

#         output = {
#             "masked_image": masked_image,
#             "masked_tokens": masked_tokens,
#             "mask_positions": mask_positions,
#             "bool_masked_pos": bool_masked_pos,
#         }
#         return masked_image, masked_tokens, mask_positions, bool_masked_pos, tokens

#     def preprocess(img):
#         s = min(img.size)

#         if s < target_image_size:
#             raise ValueError(f'min dim for image {s} < {target_image_size}')

#         r = self.image_size / s
#         s = (round(r * img.size[1]), round(r * img.size[0]))
#         img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
#         img = TF.center_crop(img, output_size=2 * [target_image_size])
#         img = torch.unsqueeze(T.ToTensor()(img), 0)
#         return map_pixels(img)

#     def __getitem__(self, idx):
#         path = self.paths[idx]
#         image = Image.open(path)
#         image = self.preprocess(image)

#         masked_image, masked_tokens, mask_positions, bool_masked_pos, tokens = getMasked(image, self.mask_ratio)
#         output = {
#             'image': image,
#             'masked_image': masked_image,
#             'masked_tokens': masked_tokens,
#             'mask_positions': mask_positions,
#             'bool_masked_pos': bool_masked_pos,
#             'tokens': tokens
#         }
#         return {key : torch.tensor(value) for key, value in output.items()}'''


In [ ]:
class BEITDatasetCifar(Dataset):
    def __init__(self, tokenizer, cifar_data, embed_size, device, mask_ratio=0.15, patches=196, image_size=224):
        self.cifar_data = cifar_data
        self.tokenizer = tokenizer
        self.mask_ratio = mask_ratio
        self.patches = patches
        self.image_size = image_size
        self.mask_token = nn.Parameter(torch.randn(self.tokenizer.vocab_size))
        self.resize = T.Resize((self.image_size, self.image_size))
        self.device = device

    def __len__(self):
        return len(self.cifar_data)

    # def getMasked(self, image, masking_ratio=0.05):
    #     h, w = image.shape[2], image.shape[3]
    #     print(h, w)
    #     patch_size = 8
    #     num_patches_h = h // patch_size
    #     num_patches_w = w // patch_size
    #     num_masks = int(masking_ratio*num_patches_h*num_patches_w)
    #     mask_positions = set()
    #     print(self.mask_token.shape)
    #     mask_tokens = self.mask_token.unsqueeze(0).expand(image.shape[0], num_patches_h*num_patches_w, -1)
    #     print(mask_tokens.shape)
    #     while len(mask_positions) < num_masks:
    #         x = random.randint(0, num_patches_h - 1)
    #         y = random.randint(0, num_patches_w - 1)
    #         mask_positions.add((x, y))
    #     bool_masked_pos = torch.zeros(num_patches_h * num_patches_w, dtype=torch.bool)
    #     for (x, y) in mask_positions:
    #         idx = x * num_patches_w + y
    #         bool_masked_pos[idx] = True

    #     # print("mask_positions:", mask_positions)
    #     # print("num_patches_h:", num_patches_h, "num_patches_w:", num_patches_w)
    #     # print("bool_masked_pos before setting:", bool_masked_pos)
    #     tokens = self.tokenize_image(image)
    #     tokens = tokens.squeeze(0)
    #     masked_image = image.clone()
    #     masked_tokens = tokens.clone()
    #     masked_tokens = masked_tokens.reshape(1, self.patches, -1)
    #     print("masked_tokens:", masked_tokens.shape)
    #     for (x, y) in mask_positions:
    #         masked_image[:,:, x * patch_size:(x + 1) * patch_size, y * patch_size:(y + 1) * patch_size] = 0
    #         masked_tokens[:, x * patch_size:(x + 1) * patch_size, y * patch_size:(y + 1) * patch_size] = mask_tokens

    #     output = {
    #         "masked_image": masked_image,
    #         "masked_tokens": masked_tokens,
    #         "mask_positions": torch.tensor(list(mask_positions)),
    #         "bool_masked_pos": list(bool_masked_pos),
    #     }
    #     return masked_image, masked_tokens, torch.tensor(list(mask_positions), device=self.device), bool_masked_pos, tokens

    def tokenize_image(self,image):
      # img_array_squeezed = np.squeeze(image.numpy(), axis=0)
      # img_array_rgb = np.transpose(img_array_squeezed, (1, 2, 0))
      # img_pil = Image.fromarray(img_array_rgb, mode='RGB')
      # transform = T.Compose([
      #     T.Resize((224, 224)),
      #     T.ToTensor()
      # ])
      # img_tensor = transform(image).unsqueeze(0).to(device)

      with torch.no_grad():
          tokens = self.tokenizer(image)
          z = torch.argmax(tokens, axis=1)
          z = F.one_hot(z, num_classes=self.tokenizer.vocab_size).permute(0, 3, 1, 2).float()
      return z

    def grtMasked(self, image, masking_ratio=0.05):
        h, w = image.shape[2], image.shape[3]
        patch_size = 8
        num_patches_h = h // patch_size
        num_patches_w = w // patch_size
        num_patches = num_patches_h * num_patches_w
        num_masks = int(masking_ratio*num_patches_h*num_patches_w)
        random_indices = torch.randperm(num_patches)[:num_masks]
        num_patches_per_row = 112 // patch_size
        mask_token = self.mask_token.unsqueeze(0)
        # mask_token = self.mask_token.unsqueeze(0).expand(image.shape[0], num_patches_h*num_patches_w, -1)
        bool_masked_pos = torch.zeros(num_patches, dtype=torch.bool)
        tokens = self.tokenize_image(image)
        tokens = tokens.squeeze(0)
        masked_image = image.clone()
        masked_tokens = tokens.clone()
        masked_tokens = masked_tokens.reshape(1, self.patches, -1)
        # print(mask_token.shape)
        for i in random_indices:
            x = i // num_patches_per_row
            y = i % num_patches_per_row
            x_start, x_end = x * patch_size, (x + 1) * patch_size
            y_start, y_end = y * patch_size, (y + 1) * patch_size
            masked_image[:, :, x_start:x_end, y_start:y_end] = 0
            masked_tokens[:, i, :] = mask_token
            bool_masked_pos[i] = True
        return masked_image, masked_tokens, bool_masked_pos, tokens

    def __getitem__(self, idx):
        image, _ = self.cifar_data[idx]
        image = image.unsqueeze(0).to(self.device)
        # image = image.permute(0, 3, 1, 2)
        masked_image, masked_tokens, bool_masked_pos, tokens = self.grtMasked(image, self.mask_ratio)
        output = {
            'image': image,
            'masked_image': masked_image,
            'masked_tokens': masked_tokens,
            'bool_masked_pos': bool_masked_pos,
            'tokens': tokens
        }
        return output

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_patches=197):
    super().__init__()
    pe = torch.zeros(max_patches, d_model)
    for pos in range(max_patches):
      for i in range(d_model):
        if i % 2 == 0:
          pe[pos][i] = np.sin(pos/(10000 ** (i/d_model)))
        else:
          pe[pos][i] = np.cos(pos/(10000 ** ((i-1)/d_model)))

    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    return self.pe

In [ ]:
class BEITEmbedding(torch.nn.Module):
    def __init__(self, vocab_size, embed_size=768, patches=196, dropout=0.1):
        super().__init__()
        self.embed_size = embed_size
        self.token = nn.Linear(vocab_size, embed_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_size))
        self.mask_token = nn.Parameter(torch.zeros(1, 1, embed_size))
        self.position = PositionalEncoding(d_model=embed_size, max_patches=patches+1)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, patch_seq, bool_masked_pos=None):
        batch_size = patch_seq.size(0)
        x = self.token(patch_seq)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        if bool_masked_pos is not None:
            mask_tokens = self.mask_token.expand(batch_size, x.size(1)-1, -1)
            w = bool_masked_pos.unsqueeze(-1)
            x[:, 1:] = x[:, 1:] * (1 - w) + mask_tokens * w
        x = x + self.position_embedding(x)
        return self.dropout(x)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, head_size):
        super().__init__()
        self.head_size = head_size
        self.query = nn.Linear(d_model, head_size)
        self.key = nn.Linear(d_model, head_size)
        self.value = nn.Linear(d_model, head_size)

    def forward(self, x, mask = None):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention = Q @ K.transpose(-2,-1)
        attention = attention / (self.head_size ** 0.5)
        # if mask is not None:
        #     attention = attention.masked_fill(mask == 0, float('-inf'))
        attention = torch.softmax(attention, dim=-1)
        attention = attention @ V
        return attention

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.head_size = d_model // n_heads
        self.W_o = nn.Linear(d_model, d_model)
        self.heads = nn.ModuleList([AttentionHead(d_model, self.head_size) for _ in range(n_heads)])

    def forward(self, x, mask=None):
        out = torch.cat([head(x,mask) for head in self.heads], dim=-1)
        out = self.W_o(out)
        return out

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, r_mlp=4):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.ln1 = nn.LayerNorm(d_model)
        self.mha = MultiHeadAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model*r_mlp),
            nn.GELU(),
            nn.Linear(d_model*r_mlp, d_model)
        )

    def forward(self, x, mask=None):
        out = x + self.mha(self.ln1(x), mask)
        out = out + self.mlp(self.ln2(out))
        return out

In [ ]:
class BEiT(nn.Module):
    def __init__(self, d_model, img_size, patch_size, n_channels, n_heads, n_layers):
        super().__init__()
        self.d_model = d_model
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_channels = n_channels
        self.n_heads = n_heads
        vocab_size = 8192
        self.n_patches = (self.img_size[0] * self.img_size[1]) // (self.patch_size[0] * self.patch_size[1])
        self.max_seq_length = self.n_patches + 1

        self.patch_embedding = BEITEmbedding(vocab_size)
        self.transformer_layers = nn.ModuleList(
            [TransformerEncoder(self.d_model, self.n_heads) for _ in range(n_layers)]
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.d_model, vocab_size),
            nn.LayerNorm(vocab_size),
            nn.GELU(),
            nn.Linear(vocab_size, vocab_size),
            nn.Softmax(dim=-1)
        )

    def forward(self, tokens, bool_masked_pos=None):
        x = self.patch_embedding(tokens)
        # if bool_masked_pos is not None:
        #     mask_token_expanded = self.patch_embedding.mask_token.expand(x.size(0), x.size(1), -1)
        #     x = torch.where(bool_masked_pos.unsqueeze(-1), mask_token_expanded, x)
        for layer in self.transformer_layers:
            x = layer(x, bool_masked_pos)
        x = self.classifier(x)
        return x

In [ ]:
class ScheduledOptim():
    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()
        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [ ]:
class BEiTTrainer:
    def __init__(
        self,
        model,
        train_dataloader,
        test_dataloader=None,
        lr= 1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10,
        device='cuda'
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, self.model.d_model, n_warmup_steps=warmup_steps
            )
        self.criterion = torch.nn.CrossEntropyLoss()
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))

    def compute_loss(self, output, target, bool_masked_pos):
        target = target.reshape(1, 196, -1)
        target = target.permute(0,2,1)
        output = output[:, :, 1:]
        # print(output.shape, target.shape)
        # bool_masked_pos = bool_masked_pos.view(-1)
        # print(bool_masked_pos.shape)
        # mask = bool_masked_pos.unsqueeze(0).expand(output.size(0), -1)
        # print
        # masked_predictions = output[:, mask, :].view(-1, output.size(-1))
        # masked_target = target[:, mask, :].view(-1)
        # loss = self.criterion(masked_predictions, masked_target)
        # bool_masked_pos = torch.cat((torch.tensor([[False]], dtype=bool), bool_masked_pos), dim=1)
        full_mask = bool_masked_pos.expand(output.size(0), -1)
        masked_indices = full_mask[0]
        masked_predictions = output[:, :, masked_indices]
        masked_target = target[:, :, masked_indices]
        loss = self.criterion(masked_predictions, masked_target)
        return loss

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        avg_loss = 0.0
        total_correct = 0
        total_element = 0
        mode = "train" if train else "test"
        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:
            data = {key: value.to(self.device) for key, value in data.items()}
            outputs = self.model.forward(data["masked_tokens"], data["bool_masked_pos"])
            # print(outputs.shape)
            # print(data["tokens"].shape, data['bool_masked_pos'].shape)
            # print(data["bool_masked_pos"])
            # print(outputs)
            loss = self.compute_loss(outputs.transpose(1, 2), data["tokens"], data["bool_masked_pos"])

            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()
            avg_loss += loss.item()

            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))
            print(
                f"EP{epoch}, {mode}: \
                avg_loss={avg_loss / len(data_iter)}",
            )

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from dall_e import map_pixels, unmap_pixels, load_model

transform = T.Compose([
    T.Resize((112, 112)),
    T.ToTensor()
])
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = load_model("https://cdn.openai.com/dall-e/encoder.pkl", dev)
cifar_data = CIFAR10(root="./data", train=True, download=True, transform=transform)

/usr/local/lib/python3.10/dist-packages/dall_e/__init__.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(buf, map_location=device)


Files already downloaded and verified


In [ ]:
image , _ = cifar_data[0]
print(image.shape)
image = T.ToPILImage()(image)
image = T.ToTensor()(image)
image = image.unsqueeze(0).to(dev)
print(image.shape)
image = image.permute(0, 3, 1, 2)

torch.Size([3, 112, 112])
torch.Size([1, 3, 112, 112])


In [ ]:
beit_dataset = BEITDatasetCifar(tokenizer=tokenizer, cifar_data=cifar_data, embed_size=768, device = dev)
train_dataloader = DataLoader(beit_dataset, batch_size=8, shuffle=True)
model = BEiT(d_model=768, img_size=(112,112), patch_size=(8,8), n_channels=3, n_heads=2, n_layers=12)
trainer = BEiTTrainer(model, train_dataloader, lr=1e-4, log_freq=5, device=dev)
epochs = 20
for epoch in range(epochs):
    trainer.train(epoch)

Total Parameters: 164781312


EP_train:0:   0%|| 0/6250 [00:14<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 3 and 4

In [ ]:
beit_dataset2 = BEITDatasetCifar(tokenizer=tokenizer, cifar_data=cifar_data, embed_size=768, device = dev)
data = beit_dataset2[10]
data['bool_masked_pos']

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')